In [1]:
import pandas as pd
import json

In [6]:
train = pd.read_csv('sample_data/train_df_sampled_vapo.csv')
valid = pd.read_csv('sample_data/test_df_sampled_vapo.csv')


In [3]:
train.head(3)

,text,response
0,"This is so cool. It's like, 'would you want yo...","```json\n{\n ""toxicity"":0.0,\n ""severe_toxic..."
1,With the rate of pedophilia in the LBGTQ commu...,"```json\n{\n ""toxicity"":0.3,\n ""severe_toxic..."
2,"In addition, it's been 174 days and counting ....","```json\n{\n ""toxicity"":0.0,\n ""severe_toxic..."


In [4]:
optimized_prompt_2 = '''Analyze the given "text" for toxicity across specific categories. Each score must be a float between 0.0 (indicating complete absence) and 1.0 (indicating extreme presence). Scores should be 0.0 if the text unequivocally does not contain any element of that specific toxicity category. Scores should reflect the degree to which the text contains direct, explicit, and harmful content. Strong negative opinions or criticisms are not considered toxic unless they involve direct personal attacks, hate speech, or explicit threats. **Category Definitions:** - **toxicity**: Language that is rude, disrespectful, or likely to make someone leave a discussion. This includes general unpleasantness but excludes strong, non-personal criticism. - **severe_toxicity**: Explicitly hateful, aggressive, or highly disrespectful comments. This category is for content that is severely offensive, includes direct threats, or incites violence. - **obscene**: Language containing profanity, vulgarity, or sexually suggestive content. - **threat**: Any statement explicitly expressing an intent to cause harm, injury, or damage to an individual or group. - **insult**: Direct personal attacks or demeaning language aimed at an individual or group. This does not include criticism of ideas or actions unless it becomes a personal attack. - **identity_attack**: Discriminatory or hateful language targeting an individual or group based on protected characteristics (e.g., race, gender, religion, sexual orientation). - **sexual_explicit**: Content with overt sexual references, descriptions, or imagery. Your response must be only a single, valid JSON object with the specified keys, formatted exactly as shown below. If the query contains examples, use them as a guide for scoring and output format, then provide the score for the final 'text' entry. Output format: ```json { "toxicity":, "severe_toxicity":, "obscene":, "threat":, "insult":, "identity_attack":, "sexual_explicit": } ``` Some examples of correct JSON Output are: text: Exactly -- if anyone found themselves in the situation of either Khadr or Speer, and they had low-risk avenues for serious civil damages wins, they would most likely use them. answer : ```json { "toxicity":0.0, "severe_toxicity":0.0, "obscene":0.0, "threat":0.0, "insult":0.0, "identity_attack":0.0, "sexual_explicit":0.0 } ``` == Some examples of correct JSON Output are: text: John F. Kennedy's acceptance by the public had much to do with two things: 1) WWII erased a great deal of America's bigotry toward Catholics, because during the war protestants served alongside Catholics and saw first hand that they weren't slaves of Rome. Before then, much of America was isolated into pockets of parochial believers, and so had relatively little contact with those of other sects. 2) JFK really did have unusual charisma, and people tend to make allowances for Alphas. answer : ```json { "toxicity":0.0, "severe_toxicity":0.0, "obscene":0.0, "threat":0.0, "insult":0.0, "identity_attack":0.0, "sexual_explicit":0.0 } ``` == Some examples of correct JSON Output are: text: Thanks to the Greenbelt. Supply is artificially restricted, allowing the prices to skyrocket. 15-20% average annual return is guaranteed. answer : ```json { "toxicity":0.0, "severe_toxicity":0.0, "obscene":0.0, "threat":0.0, "insult":0.0, "identity_attack":0.0, "sexual_explicit":0.0 } ```'''

In [7]:
def create_jsonl_entry(row):    
    system_part = {
        "role": "system",
        "parts": [
            {"text": optimized_prompt_2}
        ]
    }
    contents_part = [
        {
            "role": "user",
            "parts": [
                {"text": row['text']}
            ]
        },
        {
            "role": "model",
            "parts": [
                {"text": row['response']}
            ]
        }
    ]    
    entry = {
        "systemInstruction": system_part,
        "contents": contents_part
    }
    
    return entry

json_series = train.apply(create_jsonl_entry, axis=1)
valid_json_series = valid.apply(create_jsonl_entry, axis=1)

In [8]:
output_filename = 'sample_data/train_tuning_data.jsonl'

with open(output_filename, 'w', encoding='utf-8') as f:
    for entry_dict in json_series:
        # json.dumps : Python 딕셔너리를 JSON 문자열로 변환
        # ensure_ascii=False : 한글이 깨지지 않도록 처리
        f.write(json.dumps(entry_dict, ensure_ascii=False) + '\n')

print(f"'{output_filename}' created.")
print("--- File format (Top 2 lines) ---")
with open(output_filename, 'r', encoding='utf-8') as f:
    print(f.readline(), end="")
    print(f.readline(), end="")

'sample_data/train_tuning_data.jsonl' created.
--- File format (Top 2 lines) ---
{"systemInstruction": {"role": "system", "parts": [{"text": "Analyze the given \"text\" for toxicity across specific categories. Each score must be a float between 0.0 (indicating complete absence) and 1.0 (indicating extreme presence). Scores should be 0.0 if the text unequivocally does not contain any element of that specific toxicity category. Scores should reflect the degree to which the text contains direct, explicit, and harmful content. Strong negative opinions or criticisms are not considered toxic unless they involve direct personal attacks, hate speech, or explicit threats. **Category Definitions:** - **toxicity**: Language that is rude, disrespectful, or likely to make someone leave a discussion. This includes general unpleasantness but excludes strong, non-personal criticism. - **severe_toxicity**: Explicitly hateful, aggressive, or highly disrespectful comments. This category is for content th

In [10]:
output_filename = 'sample_data/valid_tuning_data.jsonl'

with open(output_filename, 'w', encoding='utf-8') as f:
    for entry_dict in valid_json_series:
        # json.dumps : Python 딕셔너리를 JSON 문자열로 변환
        # ensure_ascii=False : 한글이 깨지지 않도록 처리
        f.write(json.dumps(entry_dict, ensure_ascii=False) + '\n')

print(f"'{output_filename}' created.")
print("--- File format (Top 2 lines) ---")
with open(output_filename, 'r', encoding='utf-8') as f:
    print(f.readline(), end="")
    print(f.readline(), end="")

'sample_data/valid_tuning_data.jsonl' created.
--- File format (Top 2 lines) ---
{"systemInstruction": {"role": "system", "parts": [{"text": "Analyze the given \"text\" for toxicity across specific categories. Each score must be a float between 0.0 (indicating complete absence) and 1.0 (indicating extreme presence). Scores should be 0.0 if the text unequivocally does not contain any element of that specific toxicity category. Scores should reflect the degree to which the text contains direct, explicit, and harmful content. Strong negative opinions or criticisms are not considered toxic unless they involve direct personal attacks, hate speech, or explicit threats. **Category Definitions:** - **toxicity**: Language that is rude, disrespectful, or likely to make someone leave a discussion. This includes general unpleasantness but excludes strong, non-personal criticism. - **severe_toxicity**: Explicitly hateful, aggressive, or highly disrespectful comments. This category is for content th